# Prototype : request, scrap data from one url

In [1]:
import requests # 어디에서-가져올까 패키지
import bs4
import pandas as pd
import time
import re
import json
from fake_useragent import UserAgent

In [3]:
vowels = "аэиоуяеыёю"
def new_agent() -> dict :
    ua = UserAgent()
    header = {
        'User-Agent': ua.random,
    }
    return header

def new_proxy() -> dict :
    proxy_url = "https://gimmeproxy.com/api/getProxy"
    while True:
        try:
            result = requests.get(proxy_url)
        except requests.exceptions.ConnectionError:
            continue

        if result.status_code != 200:
            continue

        random_proxy_byte_str = result.content
        random_proxy_dict = json.loads(random_proxy_byte_str)
        proxy_url = random_proxy_dict['curl'].replace("<br>", "")
        proxy = {'http' : proxy_url, 'https' : proxy_url}
        return proxy

def check_vowel(s : list) -> int :
    count = 0
    for c in s:
        if c in vowels:
            count += 1
    return count

# assumption : contents look like str + bs4.element.Tag + str (order doesn't matter)
def parse_forma(soup : bs4.BeautifulSoup) -> tuple : 
    tag = soup.find('strong')
    contents = tag.contents
    full_word = ""
    stress = 0
    vowel_count = 0 
    for content in contents:
        if type(content) == bs4.element.NavigableString:
            vowel_count += check_vowel(content.string)
        elif type(content) == bs4.element.Tag:
            stress = vowel_count + 1
        else:
            return pd.Series({'stress' : -1, 'word_with_accent' : "PARSE_ERROR"})
        full_word += content.string

    return pd.Series({'stress' : stress, 'word_with_accent' : full_word})
    
def get_stress_and_fullword(word : str, collected_dict : dict, header : dict, proxy : dict) :
    if word in collected_dict:
        if collected_dict[word]['word_with_accent'] not in ["CONNECTION_FAIL", "DETECTED", "REQUEST FAIL"]:
            return True, collected_dict[word]

    url = f'https://udarenieru.ru/index.php?forma={word}'
    try:
        result = requests.get(url, headers=header, proxies=proxy)
    except requests.exceptions.ConnectionError:
        collected_dict[word] = pd.Series({'stress' : -1, 'word_with_accent' : "CONNECTION_FAIL"})
        return False, collected_dict[word]

    if result.status_code == 403:
        collected_dict[word] = pd.Series({'stress' : -1, 'word_with_accent' : "DETECTED"})
        return False, collected_dict[word]
    elif result.status_code != 200:
        collected_dict[word] = pd.Series({'stress' : -1, 'word_with_accent' : "NOT_FOUND"})
        return False, collected_dict[word]

    soup = bs4.BeautifulSoup(result.content, 'html.parser')
    collected_dict[word] = parse_forma(soup)
    return False, collected_dict[word]

def find_pure_word(text : str) -> str :
    found = re.sub('[a-z]*_', '', text)
    return found

In [62]:
main_df = pd.read_csv('new_data.csv')

In [63]:
main_df.head(10)

,name,name2,freq,target_word,stress,word_with_accent
0,v_бдении,vo_бдении,2,бдении,1,Бде́нии
1,s_бдениями,s_бдениями,1,бдениями,1,Бде́ниями
2,v_бдениях,v_бдениях,1,бдениях,1,Бде́ниях
3,v_бдительности,v_бдительности,2,бдительности,1,Бди́тельности
4,k_бдительности,k_бдительности,21,бдительности,1,Бди́тельности
5,s_бдительностью,s_бдительностью,2,бдительностью,1,Бди́тельностью
6,k_бжания,k_бжания,1,бжания,-1,NOT_FOUND
7,v_бжезинке,v_бжезинке,1,бжезинке,-1,NOT_FOUND
8,k_бзикам,k_бзикам,1,бзикам,1,Бзи́кам
9,s_бзиками,s_бзиками,1,бзиками,1,Бзи́ками


In [60]:
# target_df['target_word'] = target_df['name2'].map(lambda x : find_pure_word(x))
# target_df.head(10)

In [59]:
# collected_df = pd.read_csv("data.csv")
# collected_df = collected_df.set_index('word')
# collected_dict = collected_df.to_dict('index')
# collected_df.head(10)

In [64]:
not_found_status = ['CONNECTION_FAIL', 'DETECTED', 'NONE']
df_cut = main_df.query('word_with_accent in @not_found_status')
print(f'word to find {len(df_cut.index)}')

word to find 1277


In [55]:
# target_df['stress'] = target_df['target_word'].map(lambda x : collected_dict.get(x, dict(stress = -1, word_with_accent = 'NONE'))['stress'])
# target_df['word_with_accent'] = target_df['target_word'].map(lambda x : collected_dict.get(x, dict(stress = -1, word_with_accent = 'NONE'))['word_with_accent'])

In [58]:
# target_df.to_csv("new_data.csv", index=False)

In [20]:
# print('Initialization (proxy, header)...')
# my_proxy = new_proxy()
# my_agent = new_agent()

# print('Start scrap...')
# connection_fail = 0
# for i, w in enumerate(word_df.iterrows()):
#     word = w[1][0]
#     pure_w = find_pure_word(str(word))
#     cached, word_dict = get_stress_and_fullword(pure_w, collected_dict, my_agent, my_proxy)
#     stress = word_dict['stress']
#     word_with_accent = word_dict['word_with_accent']
#     if cached:
#         print(f'{i}th iteration... word : {pure_w} (cached)')    
#     else:
#         print(f'{i}th iteration... word : {pure_w}, stress : {stress}, full : {word_with_accent}')
#     if word_with_accent in ["CONNECTION_FAIL"]:
#         connection_fail += 1
#     if word_with_accent in ["DETECTED"] or connection_fail > 50:
#         print('Find a new proxy...')
#         my_proxy = new_proxy()
#         connection_fail = 0

iteration... word : златою (cached)
6597th iteration... word : злату (cached)
6598th iteration... word : злату (cached)
6599th iteration... word : злату (cached)
6600th iteration... word : зле (cached)
6601th iteration... word : зле (cached)
6602th iteration... word : зло (cached)
6603th iteration... word : зло (cached)
6604th iteration... word : злобе (cached)
6605th iteration... word : злобе (cached)
6606th iteration... word : злобе (cached)
6607th iteration... word : злобе (cached)
6608th iteration... word : злобин (cached)
6609th iteration... word : злобина (cached)
6610th iteration... word : злобино (cached)
6611th iteration... word : злобиным (cached)
6612th iteration... word : злобностью (cached)
6613th iteration... word : злобой (cached)
6614th iteration... word : злобою (cached)
6615th iteration... word : злобу (cached)
6616th iteration... word : злобы (cached)
6617th iteration... word : зловонии (cached)
6618th iteration... word : зловонью (cached)
6619th iteration... word : 

KeyboardInterrupt: 

In [22]:
new_collected_df = pd.DataFrame.from_dict(collected_dict, orient='index', columns=['stress', 'word_with_accent'])
new_collected_df = new_collected_df.reset_index()
new_collected_df.columns = ['word', 'stress', 'word_with_accent']
new_collected_df.head(20)                                                                                                                                                                                                                                                                                                   

,word,stress,word_with_accent
0,бдении,1,Бде́нии
1,бдениями,1,Бде́ниями
2,бдениях,1,Бде́ниях
3,бдительности,1,Бди́тельности
4,бдительностью,1,Бди́тельностью
5,бжания,-1,NOT_FOUND
6,бжезинке,-1,NOT_FOUND
7,бзикам,1,Бзи́кам
8,бзиками,1,Бзи́ками
9,блаватником,-1,NOT_FOUND


In [23]:
new_collected_df.to_csv("data.csv", index=False)

# Using multi-thread

In [13]:
from threading import Thread

def crawling(id : int, collected_dict : dict, data_chunk : pd.core.frame.DataFrame) :

    print(f'[worker #{id}] initialize proxy and header...')
    my_proxy = new_proxy()
    my_agent = new_agent()

    print(f'[worker #{id}] start scrap...')
    connection_fail = 0

len(data_chunk.index) == len(collected_dict) 

    for i, w in enumerate(data_chunk.iterrows()):
        word = w[1][0]
        pure_w = find_pure_word(str(word))

        while(True):
            cached, word_dict = get_stress_and_fullword(pure_w, collected_dict, my_agent, my_proxy)
            stress = word_dict['stress']
            word_with_accent = word_dict['word_with_accent']
            if cached:
                pass # print(f'[worker #{id}] {i}th iteration... word : {pure_w} (cached)')    
            else:
                print(f'[worker #{id}] {i}th iteration... word : {pure_w}, stress : {stress}, full : {word_with_accent}')
                break
            if word_with_accent in ["CONNECTION_FAIL"]:
                connection_fail += 1
            if word_with_accent in ["DETECTED"] or connection_fail > 10:
                print('Find a new proxy...')
                my_proxy = new_proxy()
                connection_fail = 0
    print(f'[worker #{id}] finish scrap...')

def distribute_range(length : int, num : int) -> tuple :
    distribution = [(i * (length // num), (i+1) * (length // num) - 1) for i in range(num)] 
    distribution[-1] = (distribution[-1][0], distribution[-1][1] + length % num)
    return distribution


In [14]:
num_workers = int(input("Please decide the number of workers : "))

print(f'Initialization {num_workers} workers...')

distribution = distribute_range(len(word_df.index), num_workers)

workers = [Thread(target=crawling, args=(i, collected_dict, word_df.loc[distribution[i][0]:distribution[i][1], :])) for i in range(num_workers)]
print(f'Whipping {num_workers} workers...')
for worker in workers:
    worker.start()

for worker in workers:
    worker.join()

print(f'finish working...')

.. word : прикрытием, stress : -1, full : CONNECTION_FAIL
[worker #22] 488th iteration... word : прикрытию, stress : -1, full : CONNECTION_FAIL
[worker #22] 491th iteration... word : прикупе, stress : -1, full : CONNECTION_FAIL
[worker #22] 494th iteration... word : прикуриватель, stress : -1, full : CONNECTION_FAIL
[worker #22] 499th iteration... word : прилавка, stress : -1, full : CONNECTION_FAIL
[worker #22] 500th iteration... word : прилавкам, stress : -1, full : CONNECTION_FAIL
[worker #22] 501th iteration... word : прилавками, stress : -1, full : CONNECTION_FAIL
[worker #22] 511th iteration... word : прилагательными, stress : -1, full : CONNECTION_FAIL
[worker #22] 512th iteration... word : прилбицей, stress : -1, full : CONNECTION_FAIL
[worker #22] 523th iteration... word : приливами, stress : -1, full : CONNECTION_FAIL
[worker #22] 533th iteration... word : прилове, stress : -1, full : CONNECTION_FAIL
[worker #22] 534th iteration... word : приловом, stress : -1, full : CONNECT